## Weather Data Transformation

This notebook extracts raw weather data from the Bronze Delta Lake table, parses JSON payloads, and flattens nested structures for analysis. The goal is to prepare a clean, structured Silver-level dataset.


### 1. Environment Setup

Import necessary libraries, configure logging, and load environment variables for S3/MinIO access.


In [9]:
import os
import logging
import json
import pandas as pd
import pyarrow as pa
from deltalake import DeltaTable, write_deltalake
from dotenv import load_dotenv

In [10]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
load_dotenv()

True

### 2. Storage Configuration

Define paths for Bronze and Silver Delta tables and setup storage options for AWS/MinIO.


In [11]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
MINIO_ENDPOINT = "http://localhost:9000"
STORAGE_OPTIONS = {
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY_ID,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_ACCESS_KEY,
    "AWS_ENDPOINT_URL": MINIO_ENDPOINT,
    "AWS_ALLOW_HTTP": "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}
BRONZE_TABLE_PATH = "s3://weather-bronze/weather_events"
SILVER_TABLE_PATH = "s3://weather-silver/weather_observations"

### 3. Load Bronze Table

Read the raw weather events from the Bronze Delta table into a Pandas DataFrame.


In [12]:
dt = DeltaTable(BRONZE_TABLE_PATH, storage_options=STORAGE_OPTIONS)
df_bronze = dt.to_pandas()
len(df_bronze)

23800

### 4. Inspect Raw Data

Check basic info and preview the raw DataFrame to understand the schema and content.


In [13]:
df_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23800 entries, 0 to 23799
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   observation_id  23800 non-null  object             
 1   source          23800 non-null  object             
 2   raw_payload     23800 non-null  object             
 3   lat             23800 non-null  float64            
 4   lon             23800 non-null  float64            
 5   _ingested_at    23800 non-null  datetime64[us, UTC]
 6   ingest_date     23800 non-null  object             
dtypes: datetime64[us, UTC](1), float64(2), object(4)
memory usage: 1.3+ MB


In [14]:
df_bronze.head()

,observation_id,source,raw_payload,lat,lon,_ingested_at,ingest_date
0,7f93552464dbdae763ae45d4bab84e767a15a22bebf33b...,weatherbit_current,"{""app_temp"": 20.7, ""aqi"": 56, ""city_name"": ""Ha...",20.8623,106.6799,2025-11-22 12:21:55.641460+00:00,2025-11-22
1,6fa2c1553011eeb52d6046f85f6e7c5883121d152bfef0...,weatherbit_current,"{""app_temp"": 29.6, ""aqi"": 161, ""city_name"": ""Q...",10.7755,106.7021,2025-11-22 12:21:55.641460+00:00,2025-11-22
2,66774d85b77dc9c53bedacd285090e4f02170a33e697ae...,weatherbit_current,"{""app_temp"": 26.4, ""aqi"": 50, ""city_name"": ""Cà...",9.0180,105.0870,2025-11-22 12:21:55.641460+00:00,2025-11-22
3,07f4180af2a4d82a50e5511f2419536a7652f524947704...,weatherbit_current,"{""app_temp"": 20.7, ""aqi"": 46, ""city_name"": ""Pl...",14.0201,108.6355,2025-11-22 12:21:55.641460+00:00,2025-11-22
4,f4846dc3caf8f3b38f035aeb21474c0ad204f6e4eac39d...,weatherbit_current,"{""app_temp"": 26.5, ""aqi"": 102, ""city_name"": ""V...",10.4252,105.9271,2025-11-22 12:21:55.641460+00:00,2025-11-22


### 5. Parse raw payload and combine initial dataframes

Extract the JSON payload from the `raw_payload` column and expand it into separate columns.


In [15]:
payload_df = df_bronze["raw_payload"].apply(lambda x: json.loads(x)).apply(pd.Series)

In [16]:
payload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23800 entries, 0 to 23799
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   app_temp          23800 non-null  float64
 1   aqi               136 non-null    float64
 2   city_name         23800 non-null  object 
 3   clouds            23800 non-null  int64  
 4   country_code      23800 non-null  object 
 5   datetime          23800 non-null  object 
 6   dewpt             23800 non-null  float64
 7   dhi               23800 non-null  int64  
 8   dni               23800 non-null  int64  
 9   elev_angle        23800 non-null  float64
 10  ghi               23800 non-null  int64  
 11  gust              23800 non-null  float64
 12  h_angle           136 non-null    float64
 13  lat               23800 non-null  float64
 14  lon               23800 non-null  float64
 15  ob_time           136 non-null    object 
 16  pod               23800 non-null  object

In [17]:
payload_df.head()

,app_temp,aqi,city_name,clouds,country_code,datetime,dewpt,dhi,dni,elev_angle,...,wind_dir,wind_spd,place_name,city_id,station_id,azimuth,revision_status,revision_version,timestamp_local,timestamp_utc
0,20.7,56.0,Haiphong,0,VN,2025-11-22:11,12.0,0,0,-25.18,...,150,2.1,Thành phố Hải Phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29.6,161.0,Quận Một,68,VN,2025-11-22:11,23.0,0,0,-22.43,...,30,2.6,Thành phố Hồ Chí Minh,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26.4,50.0,Cà Mau,99,VN,2025-11-22:11,21.7,0,0,-20.52,...,56,2.3,Cà Mau,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20.7,46.0,Pleiku,84,VN,2025-11-22:11,18.0,0,0,-24.64,...,18,2.4,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26.5,102.0,Vĩnh Long,99,VN,2025-11-22:11,22.4,0,0,-21.60,...,21,1.4,Đồng Tháp,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# combine initial dataframes
df_silver = pd.concat([df_bronze[['observation_id', '_ingested_at', 'source']], payload_df], axis=1)

In [19]:
df_silver.head()

,observation_id,_ingested_at,source,app_temp,aqi,city_name,clouds,country_code,datetime,dewpt,...,wind_dir,wind_spd,place_name,city_id,station_id,azimuth,revision_status,revision_version,timestamp_local,timestamp_utc
0,7f93552464dbdae763ae45d4bab84e767a15a22bebf33b...,2025-11-22 12:21:55.641460+00:00,weatherbit_current,20.7,56.0,Haiphong,0,VN,2025-11-22:11,12.0,...,150,2.1,Thành phố Hải Phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6fa2c1553011eeb52d6046f85f6e7c5883121d152bfef0...,2025-11-22 12:21:55.641460+00:00,weatherbit_current,29.6,161.0,Quận Một,68,VN,2025-11-22:11,23.0,...,30,2.6,Thành phố Hồ Chí Minh,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66774d85b77dc9c53bedacd285090e4f02170a33e697ae...,2025-11-22 12:21:55.641460+00:00,weatherbit_current,26.4,50.0,Cà Mau,99,VN,2025-11-22:11,21.7,...,56,2.3,Cà Mau,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07f4180af2a4d82a50e5511f2419536a7652f524947704...,2025-11-22 12:21:55.641460+00:00,weatherbit_current,20.7,46.0,Pleiku,84,VN,2025-11-22:11,18.0,...,18,2.4,Gia Lai,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,f4846dc3caf8f3b38f035aeb21474c0ad204f6e4eac39d...,2025-11-22 12:21:55.641460+00:00,weatherbit_current,26.5,102.0,Vĩnh Long,99,VN,2025-11-22:11,22.4,...,21,1.4,Đồng Tháp,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23800 entries, 0 to 23799
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   observation_id    23800 non-null  object             
 1   _ingested_at      23800 non-null  datetime64[us, UTC]
 2   source            23800 non-null  object             
 3   app_temp          23800 non-null  float64            
 4   aqi               136 non-null    float64            
 5   city_name         23800 non-null  object             
 6   clouds            23800 non-null  int64              
 7   country_code      23800 non-null  object             
 8   datetime          23800 non-null  object             
 9   dewpt             23800 non-null  float64            
 10  dhi               23800 non-null  int64              
 11  dni               23800 non-null  int64              
 12  elev_angle        23800 non-null  float64            
 13  g

### 6. Extract Weather Information

Select the `weather` column from the parsed payload, expand it into a DataFrame, and rename key columns for clarity.


In [21]:
weather_df = df_silver['weather'].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series).rename(
    columns={
        'description': 'weather_description',
        'code': 'weather_code',
        'icon': 'weather_icon'}
)
df_silver = pd.concat([df_silver, weather_df], axis=1)
df_silver = df_silver.drop(columns=['weather'])

In [22]:
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23800 entries, 0 to 23799
Data columns (total 51 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   observation_id       23800 non-null  object             
 1   _ingested_at         23800 non-null  datetime64[us, UTC]
 2   source               23800 non-null  object             
 3   app_temp             23800 non-null  float64            
 4   aqi                  136 non-null    float64            
 5   city_name            23800 non-null  object             
 6   clouds               23800 non-null  int64              
 7   country_code         23800 non-null  object             
 8   datetime             23800 non-null  object             
 9   dewpt                23800 non-null  float64            
 10  dhi                  23800 non-null  int64              
 11  dni                  23800 non-null  int64              
 12  elev_angle        

### 8. Select Relevant Columns

Identify and retain only the columns needed for the Silver layer, dropping unnecessary or redundant fields.


In [23]:
final_silver_columns = [
    'id', 'source', 'ingested_at',
    'place', 'api_city', 'country', 'latitude', 'longitude',
    'observed_at',
    'temperature', 'apparent_temperature',
    'humidity', 'pressure', 'sea_level_pressure',
    'wind_speed', 'wind_direction', 'wind_gust',
    'precipitation', 'cloud_cover', 'uv_index', 'visibility', 'aqi',
    'weather_code', 'weather_description'
]

In [24]:
column_mapping = {
    "observation_id": "id",
    "_ingested_at": "ingested_at",

    "place_name": "place",
    "city_name": "api_city",
    "country_code": "country",
    "lat": "latitude",
    "lon": "longitude",

    "ts": "observed_at",

    "temp": "temperature",
    "app_temp": "apparent_temperature",

    "rh": "humidity",
    "pres": "pressure",
    "slp": "sea_level_pressure",

    "wind_spd": "wind_speed",
    "wind_dir": "wind_direction",
    "gust": "wind_gust",

    "precip": "precipitation",
    "clouds": "cloud_cover",
    "uv": "uv_index",
    "vis": "visibility",

    "aqi": "aqi",
    
    "weather_code": "weather_code",
    "weather_description": "weather_description"
}

In [25]:
df_silver.rename(columns=column_mapping, inplace=True)

for col in final_silver_columns:
    if col not in df_silver.columns:
        df_silver[col] = None

df_silver = df_silver[final_silver_columns]

In [26]:
df_silver['observed_at'] = pd.to_datetime(df_silver['observed_at'], unit='s', utc=True, errors='coerce')

### 9. Deduplicating records based on location and observation hour


In [27]:
num_rows_before = len(df_silver)
num_rows_before

23800

In [28]:
df_silver.dropna(subset=['observed_at', 'place'], inplace=True)
df_silver['observed_at']

0       2025-11-22 11:50:59+00:00
1       2025-11-22 11:51:01+00:00
2       2025-11-22 12:06:03+00:00
3       2025-11-22 12:06:05+00:00
4       2025-11-22 12:06:06+00:00
                   ...           
23795   2025-09-29 19:00:00+00:00
23796   2025-09-29 20:00:00+00:00
23797   2025-09-29 21:00:00+00:00
23798   2025-09-29 22:00:00+00:00
23799   2025-09-29 23:00:00+00:00
Name: observed_at, Length: 23800, dtype: datetime64[ns, UTC]

In [29]:
df_silver['observation_ymdh'] = df_silver['observed_at'].dt.strftime('%Y-%m-%d-%H')
df_silver['observation_ymdh']

0        2025-11-22-11
1        2025-11-22-11
2        2025-11-22-12
3        2025-11-22-12
4        2025-11-22-12
             ...      
23795    2025-09-29-19
23796    2025-09-29-20
23797    2025-09-29-21
23798    2025-09-29-22
23799    2025-09-29-23
Name: observation_ymdh, Length: 23800, dtype: object

In [30]:
df_silver.sort_values(by=['place', 'observation_ymdh', 'observed_at'], ascending=True, inplace=True)

df_silver.drop_duplicates(subset=['place', 'observation_ymdh'], keep='first', inplace=True)

df_silver.drop(columns=['observation_ymdh'], inplace=True)
num_rows_after = len(df_silver)

In [31]:
logging.info(f"Deduplication removed {num_rows_before - num_rows_after} duplicate records.")

2025-11-22 20:23:10,261 - INFO - Deduplication removed 75 duplicate records.


In [32]:
df_silver.dtypes

id                                   object
source                               object
ingested_at             datetime64[us, UTC]
place                                object
api_city                             object
country                              object
latitude                            float64
longitude                           float64
observed_at             datetime64[ns, UTC]
temperature                         float64
apparent_temperature                float64
humidity                              int64
pressure                            float64
sea_level_pressure                  float64
wind_speed                          float64
wind_direction                        int64
wind_gust                           float64
precipitation                       float64
cloud_cover                           int64
uv_index                            float64
visibility                          float64
aqi                                 float64
weather_code                    

### 10. Feature Engineering

Create derived columns like observation hour, day of the week, wind category, and UV category to enrich the dataset for analysis.


In [33]:
df_silver['observation_hour'] = df_silver['observed_at'].dt.hour
df_silver['day_of_week'] = df_silver['observed_at'].dt.day_name()
df_silver['month'] = df_silver['observed_at'].dt.month
df_silver[['observation_hour', 'day_of_week', 'month']].head()

,observation_hour,day_of_week,month
5008,0,Monday,9
5009,1,Monday,9
5010,2,Monday,9
5011,3,Monday,9
5012,4,Monday,9


In [34]:
df_silver['wind_speed'].describe()


count    23725.000000
mean         1.929494
std          1.455280
min          0.000000
25%          0.800000
50%          1.600000
75%          2.400000
max         17.930000
Name: wind_speed, dtype: float64

In [35]:
bins = [-0.1, 0.2, 1.5, 3.3, 5.5, 7.9, 10.7, 13.8]
labels = ['Calm', 'Light air', 'Light breeze', 'Gentle breeze', 'Moderate breeze', 'Fresh breeze', 'Strong breeze']
df_silver['wind_category'] = pd.cut(df_silver['wind_speed'], bins=bins, labels=labels, right=True)

uv_bins = [-0.1, 2, 5, 7, 10, 15]
uv_labels = ['Low', 'Moderate', 'High', 'Very High', 'Extreme']
df_silver['uv_category'] = pd.cut(df_silver['uv_index'], bins=uv_bins, labels=uv_labels, right=True)


In [36]:
df_silver[['wind_category', 'uv_category']].head()

,wind_category,uv_category
5008,Gentle breeze,Low
5009,Moderate breeze,Low
5010,Moderate breeze,Low
5011,Gentle breeze,Moderate
5012,Moderate breeze,Moderate


In [37]:
df_silver['wind_category'].unique()

['Gentle breeze', 'Moderate breeze', 'Light breeze', 'Light air', 'Calm', 'Fresh breeze', 'Strong breeze', NaN]
Categories (7, object): ['Calm' < 'Light air' < 'Light breeze' < 'Gentle breeze' < 'Moderate breeze' < 'Fresh breeze' < 'Strong breeze']

In [38]:
df_silver['uv_category'].unique()

['Low', 'Moderate', 'High', 'Very High', 'Extreme']
Categories (5, object): ['Low' < 'Moderate' < 'High' < 'Very High' < 'Extreme']

In [39]:
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23725 entries, 5008 to 4
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   id                    23725 non-null  object             
 1   source                23725 non-null  object             
 2   ingested_at           23725 non-null  datetime64[us, UTC]
 3   place                 23725 non-null  object             
 4   api_city              23725 non-null  object             
 5   country               23725 non-null  object             
 6   latitude              23725 non-null  float64            
 7   longitude             23725 non-null  float64            
 8   observed_at           23725 non-null  datetime64[ns, UTC]
 9   temperature           23725 non-null  float64            
 10  apparent_temperature  23725 non-null  float64            
 11  humidity              23725 non-null  int64              
 12  pressure  

### 10. Write to Silver Delta Table

Convert the cleaned and enriched DataFrame to a PyArrow Table and write it to the Silver Delta table on S3/MinIO, overwriting previous versions if necessary.


In [40]:
arrow_table = pa.Table.from_pandas(df_silver, preserve_index=False)

write_deltalake(
    SILVER_TABLE_PATH,
    arrow_table,
    mode="overwrite",
    storage_options=STORAGE_OPTIONS
)